In [1]:
import pandas as pd
import sys

In [2]:
sys.path.append(r'c:\\Users\\giovanni.5683\\GITHUB\\controlling_postgreSQL')
from conn import get_connection # type: ignore

conn = get_connection()
cursor = conn.cursor()

In [6]:
conn.rollback()

# Ler o CSV
df = pd.read_csv(r'c:\Users\giovanni.5683\GITHUB\controlling_postgreSQL\Datasets\dpto.csv')

# Function to convert Brazilian format to numeric
def convert_br_to_numeric(value):
    if isinstance(value, str):
        # Remove dots (thousand separator) and replace comma with dot (decimal separator)
        value = value.replace('.', '').replace(',', '.')
    try:
        return float(value)
    except:
        return None

# Convert numeric columns from Brazilian format
df['bonus_dpto'] = df['bonus_dpto'].apply(convert_br_to_numeric)
df['trade_mkt_dpto'] = df['trade_mkt_dpto'].apply(convert_br_to_numeric)
df['rebate_dpto'] = df['rebate_dpto'].apply(convert_br_to_numeric)
df['ipva_dpto'] = df['ipva_dpto'].apply(convert_br_to_numeric)

# Atualizar a tabela usando UPDATE com base no idnfsexterno
for index, row in df.iterrows():
    # Skip rows where all target columns are NaN
    if pd.isna(row['bonus_dpto']) and pd.isna(row['trade_mkt_dpto']) and pd.isna(row['rebate_dpto']) and pd.isna(row['ipva_dpto']):
        continue
    
    idnfsexterno = str(row['idnfsexterno']).strip()  # Remove espaços extras
    print(f"Atualizando idnfsexterno: {idnfsexterno}")  # Debug
    query = """
    UPDATE byd.controladoria 
    SET bonus_dpto = %s,
        trade_mkt_dpto = %s,
        rebate_dpto = %s,
        ipva_dpto = %s
    WHERE idnfsexterno = %s
    """
    cursor.execute(query, (
        row['bonus_dpto'], 
        row['trade_mkt_dpto'], 
        row['rebate_dpto'], 
        row['ipva_dpto'], 
        idnfsexterno
    ))
    print(f"Registros afetados nesta linha: {cursor.rowcount}")  # Debug

conn.commit()
print(f"{cursor.rowcount} registros atualizados com sucesso!")

Atualizando idnfsexterno: A25331121V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25414541V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25622391V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25514421V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25738931V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25333481V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25333371V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A251227631V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A2582341V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25514361V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25334241V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A2582201V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A25740271V210
Registros afetados nesta linha: 1
Atualizando idnfsexterno: A251224481V21

In [7]:
# Consultar todos os dados da tabela byd.controladoria e criar um DataFrame para visualização
import pandas as pd

query = "SELECT * FROM byd.controladoria"
df_controladoria = pd.read_sql(query, conn)
df_controladoria.head()

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_29748\1712869169.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_controladoria = pd.read_sql(query, conn)


,idnfsexterno,chassi,revenda,empresa,des_modelo,modelo_format,proposta,valor_venda_bruta,val_custo_contabil,val_modalidade,...,bonus_utilizado,bonus_recebido,dta_baixa_contabil,num_nd,prejuizo,bonus_dpto,trade_mkt_dpto,rebate_dpto,ipva_dpto,detalhes_dpto
0,A25261751V210,LGXCD4C45T0042033,2,25,BYD SONG PLUS PREMIUM DM-I 1.5T 135km(L8) - S...,BYD SONG PLUS PREMIUM,4288,259990.000000000,275816.000000000,1953.300000000,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None
1,A251289111V210,LC0CE4CC2T0035144,1,25,DOLPHIN 70KW Motor 405km - Delan Black - Boba ...,BYD DOLPHIN,17437,149990.000000000,140990.600000000,3194.400000000,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None
2,A251288911V210,92VCE4CC0TK010946,1,25,DOLPHIN MINI 5Seats 380km(Brazil SKD-2) - Obs...,BYD DOLPHIN MINI,17170,119990.000000000,112790.600000000,1606.500000000,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None
3,A25525591V210,92VCE4CC8TK001959,5,25,DOLPHIN MINI 5Seats 380km(Brazil SKD-2) - Apr...,BYD DOLPHIN MINI,1808,119740.000000000,112790.600000000,940.000000000,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None
4,A25261771V210,LGXC74C4XT0054329,2,25,BYD SONG PLUS DM-i 1.5L 100km(L8) - Snow Whit...,BYD SONG PLUS,4263,249990.000000000,229990.800000000,17355.000000000,...,None,None,None,None,None,NaN,NaN,NaN,NaN,None


In [8]:
dpto_query = """ SELECT * FROM byd.bonus_view"""
df_bonus = pd.read_sql(dpto_query, conn)
df_bonus

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_29748\1430589873.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bonus = pd.read_sql(dpto_query, conn)


,idnfsexterno,revenda,empresa,numero_nota_fiscal,serie_nota_fiscal,dta_processamento,tipo_transacao,des_tipo_transacao,cod_cliente,nome_cliente,...,opt_comecial,campanha,detalhes,bonus_utilizado,bonus_dpto,rebate_dpto,ipva_dpto,trade_mkt_dpto,data_final_venda,competencia
0,A25254761V210,2,25,5476,1,2025-10-27 12:10:04,V21,VENDA DE VEICULOS NOVOS,3784234,ADILSON FERRASINI,...,Opção 2,214/25,None,Trade_in,15000.0,NaN,NaN,NaN,2025-10-27 12:10:04,Outubro
1,A25340061V210,3,25,4006,1,2025-10-28 18:01:27,V21,VENDA DE VEICULOS NOVOS,1264164,RENATO RODRIGUES DA SILVA,...,Não se Aplica,214/25,None,Sem Bônus,NaN,NaN,NaN,NaN,2025-10-28 18:01:27,Outubro
2,A25754151V210,7,25,5415,1,2025-11-14 11:31:06,V21,VENDA DE VEICULOS NOVOS,3790955,ALYNE SGOBERO DEPIERI,...,Não se Aplica,224/25,None,Sem Bônus,0.0,NaN,NaN,0.0,2025-11-14 11:31:06,Novembro
3,A251150561V210,1,25,15056,1,2025-02-05 09:56:17,V21,VENDA DE VEICULOS NOVOS,36945,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,None,None,None,Varejo,NaN,NaN,NaN,NaN,2025-02-05 09:56:17,Fevereiro
4,A25338861V210,3,25,3886,1,2025-10-15 11:36:26,V21,VENDA DE VEICULOS NOVOS,1642664,AUTOPLUS COMERCIO DE VEICULOS LTDA,...,Opção 1,197/25 - 214/25,BÔNUS EMPLACAMENTO,Equalização,14221.4,NaN,NaN,NaN,2025-10-15 11:36:26,Outubro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4679,A25249051V210,2,25,4905,1,2025-09-17 13:10:19,V21,VENDA DE VEICULOS NOVOS,3773133,LUIS AUGUSTO RICCIARDI SORDI,...,Opção 1,198/25,None,"Seguro, Trade_in",5000.0,NaN,NaN,NaN,2025-09-17 13:10:19,Setembro
4680,A25343781V210,3,25,4378,1,2025-11-29 15:01:02,V21,VENDA DE VEICULOS NOVOS,3830664,FERNANDA DA CRUZ DIANA,...,Não se Aplica,224/25,None,Sem Bônus,0.0,NaN,NaN,NaN,2025-11-29 15:01:02,Novembro
4681,A25344101V210,3,25,4410,1,2025-11-29 19:39:34,V21,VENDA DE VEICULOS NOVOS,3839427,LORELEY MOTTER KIKUTI,...,Não se Aplica,None,None,Sem Bônus,0.0,NaN,NaN,NaN,2025-11-29 19:39:34,Novembro
4682,A2584951V210,8,25,495,1,2025-11-26 11:48:52,V21,VENDA DE VEICULOS NOVOS,692337,J R FAVRETTO ADM IMOBILIARIA LTDA,...,Opção 1,224/2025,None,Taxa 0,0.0,NaN,NaN,NaN,2025-11-26 11:48:52,Novembro


In [9]:
conn.rollback()